In [1]:
## In this notebook, we will create a much simpler parallel simulator 
## while I take the time to properly learn parallel programming.
## Specifically, we will create a function that abstracts the functions we care about:
## merge, an image function, split, etc.
## But at the same time, allows us to play around with various timing such as the p and s 
## parameters for Amdahl's Law.

In [1]:
import numpy as np
import time
import timeit
import multiprocessing as mp

In [2]:
## Time functions
def finalize(p, parallel_time):
    total_time = parallel_time / (1 - p)
    non_parallel = total_time*p
    time.sleep(non_parallel)
    

In [16]:
## Parallel task:
'''
1. Get a list of integers
2. Add 42 to each list of integers
3. For every add, wait a certain amount of time
4. return nothing
'''

def task_nonparallel(data, sleep_time):
    for i in range(len(data)):
        time.sleep(sleep_time)
        data[i] += 42  
    return

def split(data, n):
    if n > len(data): 
        n = len(data)
    avg = len(data) / float(n)
    out = []
    last = 0.0

    while last < len(data):
        out.append(data[int(last):int(last + avg)])
        last += avg
    return out

def task_parallel(data, sleep_time, num_processors):
    num_workers = mp.cpu_count() - 1
    if (num_workers > num_processors):
        num_workers = num_processors
    jobs = []
    data = split(data, num_workers)
    for worker in range(len(data)):
        sub_data = data[worker]
        p = mp.Process(target = task_nonparallel, args = (sub_data, sleep_time))
        try:
            p.start()
        except:
            p.terminate()
            raise
        jobs.append(p)
    # make sure all jobs end
    for job in jobs:
        job.join()
        job.terminate()
    return


    

In [17]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from timeit import default_timer as timer


In [13]:
def calculate_speedup(data, sleep_time, num_processors):
    start = timer()
    task_nonparallel(data, sleep_time)
    end = timer()
    non_parallel = end - start
    
    start = timer()
    task_parallel(data, sleep_time, num_processors)
    end = timer()
    parallel = end - start
    
    return non_parallel/float(parallel)
    

In [19]:
data = [0]*100000
cores = []
speed = []
for i in range(1, 4):
    print("Speedup with " + str(i) + " core(s):")
    s = calculate_speedup(data, .00001, i)
    print(s)
    cores.append(i)
    speed.append(s)
    

Speedup with 1 core(s):
1.0870005015392565
Speedup with 2 core(s):
2.161112825162861
Speedup with 3 core(s):
2.963406610550449


In [31]:

trace0 = go.Scatter(
    x = cores,
    y = speed,
    mode = 'lines',
    name = 'nonthreaded'
)

data = [trace0]
layout = go.Layout(
    title='Relative Speedup vs Number of Cores',
    xaxis=dict(
        title='Cores',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Speedup (Nonparallel Time / Parallel Time)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
init_notebook_mode(connected=True)

fig = go.Figure(data=data, layout=layout)
iplot(fig)
    

In [33]:
def calculate_S_latency(p, data, sleep_time, num_processors):
    s = calculate_speedup(data, sleep_time, num_processors)
    speedup_latency = float(1/(1 - p + p/s))
    
    return speedup_latency
    

In [35]:
data = [0]*100000

cores = []
s = []
p = .5
for i in range(1, 4):
    print("Speedup in Latency with " + str(i) + " core(s):")
    s_l = calculate_S_latency(p, data, .00001, i)
    print(s_l)
    cores.append(i)
    s.append(s_l)

trace0 = go.Scatter(
    x = cores,
    y = s,
    mode = 'lines',
    name = 'p = .6'
)

cores = []
s = []
p = .7
for i in range(1, 4):
    print("Speedup in Latency with " + str(i) + " core(s):")
    s_l = calculate_S_latency(p, data, .00001, i)
    print(s_l)
    cores.append(i)
    s.append(s_l)

trace1 = go.Scatter(
    x = cores,
    y = s,
    mode = 'lines',
    name = 'p = .7'
)

cores = []
s = []
p = .8
for i in range(1, 4):
    print("Speedup in Latency with " + str(i) + " core(s):")
    s_l = calculate_S_latency(p, data, .00001, i)
    print(s_l)
    cores.append(i)
    s.append(s_l)

trace2 = go.Scatter(
    x = cores,
    y = s,
    mode = 'lines',
    name = 'p = .8'
)

cores = []
s = []
p = .9
for i in range(1, 4):
    print("Speedup in Latency with " + str(i) + " core(s):")
    s_l = calculate_S_latency(p, data, .00001, i)
    print(s_l)
    cores.append(i)
    s.append(s_l)
    
trace3 = go.Scatter(
    x = cores,
    y = s,
    mode = 'lines',
    name = 'p = .9'
)

data = [trace0, trace1, trace2, trace3]
layout = go.Layout(
    title='Speedup in Latency vs Number of Cores',
    xaxis=dict(
        title='Cores',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='S_Latency (1/(1 - p + p/s))',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
init_notebook_mode(connected=True)

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Speedup in Latency with 1 core(s):
1.076665374589566
Speedup in Latency with 2 core(s):
1.3506881651173324
Speedup in Latency with 3 core(s):
1.4962461582918796
Speedup in Latency with 1 core(s):
1.0530292288656804
Speedup in Latency with 2 core(s):
1.57933345670915
Speedup in Latency with 3 core(s):
1.8679544585459826
Speedup in Latency with 1 core(s):
1.0549230855231528
Speedup in Latency with 2 core(s):
1.731409138399205
Speedup in Latency with 3 core(s):
2.1300990777281212
Speedup in Latency with 1 core(s):
1.0535577114281909
Speedup in Latency with 2 core(s):
1.8973387810971465
Speedup in Latency with 3 core(s):
2.4755916958935815
